In [ ]:
!pip install langchain langchain-community langchain-core
!pip install chromadb
!pip install langchain-chroma
!pip3 install scikit-learn scipy matplotlib
!pip3 install -U sentence-transformers
!pip3 install -qU langchain-huggingface

In [54]:
!pip3 install gradio

   ---------------------------------------- 0.0/57.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/57.2 MB 4.2 MB/s eta 0:00:14
    --------------------------------------- 1.3/57.2 MB 4.0 MB/s eta 0:00:15
   - -------------------------------------- 2.1/57.2 MB 4.1 MB/s eta 0:00:14
   -- ------------------------------------- 3.1/57.2 MB 4.3 MB/s eta 0:00:13
   --- ------------------------------------ 4.5/57.2 MB 4.5 MB/s eta 0:00:12
   --- ------------------------------------ 5.5/57.2 MB 4.6 MB/s eta 0:00:12
   ---- ----------------------------------- 6.3/57.2 MB 4.5 MB/s eta 0:00:12
   ----- ---------------------------------- 7.3/57.2 MB 4.6 MB/s eta 0:00:11
   ----- ---------------------------------- 8.4/57.2 MB 4.6 MB/s eta 0:00:11
   ------ --------------------------------- 9.4/57.2 MB 4.7 MB/s eta 0:00:11
   ------- -------------------------------- 10.5/57.2 MB 4.7 MB/s eta 0:00:10
   -------- ------------------------------- 11.5/57.2 MB 4.7 MB/s eta 0:00:10
   -

  You can safely remove it manually.


In [44]:
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


In [22]:
from langchain_chroma import Chroma
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [31]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
default_ef = embedding_functions.DefaultEmbeddingFunction()
client = chromadb.PersistentClient(path='./chroma_db')
collection = client.get_collection(name="langchain",embedding_function=default_ef)

In [ ]:
from langchain_community.vectorstores import Chroma

In [38]:
# from sklearn.decomposition import PCA
# import numpy as np

# def transform_embeddings(embeddings, target_dim):
#     pca = PCA(n_components=target_dim)
#     transformed_embeddings = pca.fit_transform(embeddings)
#     return transformed_embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

chroma_db = Chroma(client=client, collection_name="langchain",embedding_function=embeddings) 

In [39]:
retriever = chroma_db.as_retriever()

In [ ]:
retriever.get_relevant_documents("what is effusion")

In [45]:
model_local = ChatOllama(model="llama3.2")

C:\Users\andre\AppData\Local\Temp\ipykernel_15992\2362477809.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model_local = ChatOllama(model="llama3.2")


In [60]:
print("\n########\nAfter RAG\n")
after_rag_template = """You are a Medical Chatbot answering questions strictly related to chest diseases. Do not provide any consultation. Give answers only for medical queries. Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("what is hotwheels?"))


########
After RAG

Unfortunately, the provided text does not mention "Hot Wheels" at all. It appears to be a passage about yoga. If you could provide more context or information about where you encountered the term "Hot Wheels", I'd be happy to try and help you find an answer!


In [58]:
import gradio as gr
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter

def process_input(question):
    model_local = ChatOllama(model="llama3.2")
    
    # Convert string of URLs to list
    # urls_list = urls.split("\n")
    # docs = [WebBaseLoader(url).load() for url in urls_list]
    # docs_list = [item for sublist in docs for item in sublist]
    
    # text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
    # doc_splits = text_splitter.split_documents(docs_list)

    # vectorstore = Chroma.from_documents(
    #     documents=doc_splits,
    #     collection_name="rag-chroma",
    #     embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
    # )
    # retriever = vectorstore.as_retriever()

    after_rag_template = """You are a Medical Chatbot answering questions strictly related to chest diseases. Do not provide any consultation. Do not mention the text from where the answer is generated. Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
    after_rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | after_rag_prompt
        | model_local
        | StrOutputParser()
    )
    return after_rag_chain.invoke(question)

# Define Gradio interface
iface = gr.Interface(fn=process_input,
                     inputs=[gr.Textbox(label="Enter query")],
                     outputs="text",
                     title="Document Query with Ollama",
                     description="enter question to query document.")
iface.launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
